In [2]:
# Install required packages if not already installed
%pip install opendatasets pandas numpy legacy-cgi

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Enhanced TroTro Multi-City Dataset Cleaning and Repackaging
import opendatasets as od
import pandas as pd
import numpy as np
import os
import warnings
from datetime import datetime
import re
import shutil
from pathlib import Path

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

print("=== TroTro Multi-City Dataset Cleaning and Analysis ===")
print(f"Starting analysis at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# 1. Download dataset with error handling
def download_dataset():
    """Download the TroTro dataset from Kaggle"""
    try:
        dataset_url = 'https://www.kaggle.com/datasets/godfredaddaiamoako/trotro'
        print("Downloading dataset...")
        od.download(dataset_url)
        print("✓ Dataset downloaded successfully")
    except Exception as e:
        print(f"⚠ Error downloading dataset: {e}")
        print("Please ensure you have Kaggle credentials configured")
        return False
    return True

# 2. Enhanced data cleaning function
def clean_data(df, filename="", city=""):
    """Comprehensive data cleaning function"""
    print(f"\nCleaning data for {city}/{filename}...")
    original_shape = df.shape
    
    if df.empty:
        print(f"  ⚠ Warning: {filename} is empty, skipping...")
        return None
    
    # Create a copy to avoid modifying original
    df_clean = df.copy()
    
    # 1. Remove completely empty rows and columns
    df_clean = df_clean.dropna(how='all').dropna(axis=1, how='all')
    
    # 2. Clean column names
    df_clean.columns = df_clean.columns.str.strip().str.lower()
    df_clean.columns = df_clean.columns.str.replace(' ', '_').str.replace(r'[^\w]', '_', regex=True)
    
    # 3. Handle duplicates
    initial_rows = len(df_clean)
    df_clean = df_clean.drop_duplicates()
    duplicates_removed = initial_rows - len(df_clean)
    if duplicates_removed > 0:
        print(f"  - Removed {duplicates_removed} duplicate rows")
    
    # 4. Clean text columns
    text_columns = df_clean.select_dtypes(include=['object']).columns
    for col in text_columns:
        if col in df_clean.columns:
            # Strip whitespace and handle common issues
            df_clean[col] = df_clean[col].astype(str).str.strip()
            df_clean[col] = df_clean[col].replace(['nan', 'NaN', 'None', ''], np.nan)
            
            # Clean special characters and normalize text
            df_clean[col] = df_clean[col].str.replace(r'\s+', ' ', regex=True)
            
    # 5. Handle missing values intelligently
    missing_threshold = 0.7  # Drop columns with >70% missing data
    columns_to_drop = []
    for col in df_clean.columns:
        missing_pct = df_clean[col].isnull().sum() / len(df_clean)
        if missing_pct > missing_threshold:
            columns_to_drop.append(col)
            print(f"  - Dropped column '{col}' (>{missing_threshold*100}% missing)")
    
    if columns_to_drop:
        df_clean = df_clean.drop(columns=columns_to_drop)
    
    # 6. Fill remaining missing values based on data type
    for col in df_clean.columns:
        if df_clean[col].isnull().any():
            if df_clean[col].dtype in ['int64', 'float64']:
                # For numeric columns, use median
                df_clean[col] = df_clean[col].fillna(df_clean[col].median())
            else:
                # For categorical columns, use mode or 'Unknown'
                mode_val = df_clean[col].mode()
                if len(mode_val) > 0:
                    df_clean[col] = df_clean[col].fillna(mode_val[0])
                else:
                    df_clean[col] = df_clean[col].fillna('Unknown')
    
    # 7. Detect and handle potential date columns
    potential_date_cols = [col for col in df_clean.columns if 'date' in col.lower() or 'time' in col.lower()]
    for col in potential_date_cols:
        try:
            df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')
            print(f"  - Converted '{col}' to datetime")
        except:
            pass
    
    # 8. Clean numeric columns
    numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        if len(df_clean[col].dropna()) > 0:  # Only if we have numeric data
            # Remove outliers using IQR method
            Q1 = df_clean[col].quantile(0.25)
            Q3 = df_clean[col].quantile(0.75)
            IQR = Q3 - Q1
            
            if IQR > 0:  # Only apply if there's variation in the data
                lower_bound = Q1 - 1.5 * IQR
                upper_bound = Q3 + 1.5 * IQR
                
                outliers_before = len(df_clean[(df_clean[col] < lower_bound) | (df_clean[col] > upper_bound)])
                df_clean[col] = df_clean[col].clip(lower=lower_bound, upper=upper_bound)
                if outliers_before > 0:
                    print(f"  - Capped {outliers_before} outliers in '{col}'")
    
    # 9. Add city identifier
    df_clean['city'] = city
    df_clean['source_file'] = filename
    
    print(f"  ✓ Cleaned data shape: {original_shape} → {df_clean.shape}")
    return df_clean

# 3. Data analysis function
def analyze_data(df, filename="", city=""):
    """Perform comprehensive data analysis"""
    print(f"\n--- Analysis for {city}/{filename} ---")
    if df is None or df.empty:
        print("No data to analyze")
        return df
        
    print(f"Shape: {df.shape}")
    print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # Missing values analysis
    missing_data = df.isnull().sum()
    if missing_data.sum() > 0:
        print("\nMissing values:")
        print(missing_data[missing_data > 0])
    else:
        print("\n✓ No missing values!")
    
    # Data types
    print(f"\nData types:")
    print(df.dtypes.value_counts())
    
    # Numeric columns summary
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        print(f"\nNumeric columns summary:")
        print(df[numeric_cols].describe())
    
    # Categorical columns info
    categorical_cols = df.select_dtypes(include=['object']).columns
    if len(categorical_cols) > 0:
        print(f"\nCategorical columns (top 3 each):")
        for col in categorical_cols[:3]:  # Limit to first 3 to avoid clutter
            print(f"\n{col}:")
            print(df[col].value_counts().head(3))
    
    # Display sample
    print(f"\nSample data (first 3 rows):")
    print(df.head(3))
    
    return df

# 4. Load data with multiple separator attempts
def load_data_file(file_path):
    """Try to load a data file with different separators"""
    file_extension = os.path.splitext(file_path)[1].lower()
    
    separators_to_try = [',', '\t', ';', '|', r'\s+']
    
    for sep in separators_to_try:
        try:
            if sep == r'\s+':
                df = pd.read_csv(file_path, sep=sep, engine='python')
            else:
                df = pd.read_csv(file_path, sep=sep)
            
            # Check if data loaded properly (more than just headers)
            if not df.empty and len(df.columns) > 1:
                print(f"  ✓ Loaded with separator '{sep}': {df.shape}")
                return df
        except Exception as e:
            continue
    
    print(f"  ❌ Could not load {file_path} with any separator")
    return None

# 5. Create output directories
def create_output_structure():
    """Create clean directory structure"""
    # Create main output directory
    output_dir = Path('cleaned_trotro_data')
    output_dir.mkdir(exist_ok=True)
    
    # Create subdirectories
    (output_dir / 'by_city').mkdir(exist_ok=True)
    (output_dir / 'combined').mkdir(exist_ok=True)
    (output_dir / 'reports').mkdir(exist_ok=True)
    
    return output_dir

# 6. Main execution function
def main():
    """Main execution function for multi-city cleaning"""
    
    # Download dataset
    if not download_dataset():
        print("Failed to download dataset. Please check your Kaggle credentials.")
        return
    
    # Find data directory structure
    base_data_dir = Path('trotro')
    if not base_data_dir.exists():
        # Try alternative directory names
        possible_dirs = [d for d in Path('.').iterdir() if d.is_dir() and 'trotro' in d.name.lower()]
        if possible_dirs:
            base_data_dir = possible_dirs[0]
        else:
            print("Could not find dataset directory")
            return
    
    # Look for the nested structure
    datasets_dir = base_data_dir / 'trotrolive-datasets'
    if not datasets_dir.exists():
        datasets_dir = base_data_dir
    
    print(f"Using data directory: {datasets_dir}")
    
    # Create output structure
    output_dir = create_output_structure()
    
    # Find all city directories
    city_dirs = [d for d in datasets_dir.iterdir() if d.is_dir()]
    print(f"Found {len(city_dirs)} city directories: {[d.name for d in city_dirs]}")
    
    if not city_dirs:
        print("No city directories found")
        return
    
    # Storage for all cleaned data
    all_city_data = {}
    all_files_data = []
    
    # Process each city
    for city_dir in city_dirs:
        city_name = city_dir.name
        print(f"\n{'='*60}")
        print(f"PROCESSING CITY: {city_name.upper()}")
        print(f"{'='*60}")
        
        # Find data files in city directory
        data_files = []
        for ext in ['*.txt', '*.csv']:
            data_files.extend(city_dir.glob(ext))
        
        if not data_files:
            print(f"No data files found in {city_name}")
            continue
            
        print(f"Found {len(data_files)} data files in {city_name}: {[f.name for f in data_files]}")
        
        # Storage for this city's data
        city_cleaned_data = {}
        
        # Process each file in the city
        for data_file in data_files:
            try:
                print(f"\n{'-'*40}")
                print(f"Processing: {city_name}/{data_file.name}")
                print(f"{'-'*40}")
                
                # Load data
                df = load_data_file(data_file)
                if df is None:
                    continue
                
                # Clean data
                df_clean = clean_data(df, data_file.name, city_name)
                if df_clean is None:
                    continue
                
                # Analyze cleaned data
                df_analyzed = analyze_data(df_clean, data_file.name, city_name)
                
                # Save individual cleaned file
                city_output_dir = output_dir / 'by_city' / city_name
                city_output_dir.mkdir(exist_ok=True)
                
                cleaned_filename = city_output_dir / f'cleaned_{data_file.stem}.csv'
                df_clean.to_csv(cleaned_filename, index=False)
                print(f"✓ Saved to: {cleaned_filename}")
                
                # Store for city-level combination
                city_cleaned_data[data_file.name] = df_clean
                
                # Store for global combination
                all_files_data.append(df_clean)
                
            except Exception as e:
                print(f"❌ Error processing {city_name}/{data_file.name}: {e}")
                continue
        
        # Combine all files for this city
        if city_cleaned_data:
            print(f"\n{'-'*40}")
            print(f"COMBINING DATA FOR {city_name.upper()}")
            print(f"{'-'*40}")
            
            try:
                # Find common columns across all files in this city
                all_columns = [set(df.columns) for df in city_cleaned_data.values()]
                common_columns = set.intersection(*all_columns) if all_columns else set()
                
                if len(common_columns) > 2:  # More than just city and source_file
                    print(f"Found {len(common_columns)} common columns for {city_name}")
                    
                    # Combine city data
                    city_combined = pd.concat([df[list(common_columns)] for df in city_cleaned_data.values()], 
                                            ignore_index=True)
                    
                    # Save city combined data
                    city_combined_file = output_dir / 'by_city' / f'{city_name}_combined.csv'
                    city_combined.to_csv(city_combined_file, index=False)
                    print(f"✓ City combined data saved: {city_combined_file}")
                    
                    # Store for global analysis
                    all_city_data[city_name] = city_combined
                    
                else:
                    print(f"⚠ Too few common columns to combine {city_name} data")
                    
            except Exception as e:
                print(f"❌ Error combining {city_name} data: {e}")
    
    # Create global combined dataset
    if all_files_data:
        print(f"\n{'='*60}")
        print("CREATING GLOBAL COMBINED DATASET")
        print(f"{'='*60}")
        
        try:
            # Find common columns across ALL files from ALL cities
            all_columns = [set(df.columns) for df in all_files_data]
            global_common_columns = set.intersection(*all_columns) if all_columns else set()
            
            if len(global_common_columns) > 2:
                print(f"Found {len(global_common_columns)} common columns across all cities")
                
                # Create global combined dataset
                global_combined = pd.concat([df[list(global_common_columns)] for df in all_files_data], 
                                          ignore_index=True)
                
                # Save global combined data
                global_combined_file = output_dir / 'combined' / 'all_cities_combined.csv'
                global_combined.to_csv(global_combined_file, index=False)
                print(f"✓ Global combined data saved: {global_combined_file}")
                
                # Generate summary report
                summary_report = f"""
TroTro Dataset Cleaning Summary Report
Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

Cities Processed: {len(all_city_data)}
Total Files Processed: {len(all_files_data)}
Global Combined Dataset Shape: {global_combined.shape}

City Breakdown:
"""
                for city, data in all_city_data.items():
                    summary_report += f"- {city}: {data.shape[0]} records, {data.shape[1]} columns\n"
                
                summary_report += f"\nCommon Columns Across All Data:\n"
                for col in sorted(global_common_columns):
                    summary_report += f"- {col}\n"
                
                # Save summary report
                report_file = output_dir / 'reports' / 'cleaning_summary.txt'
                with open(report_file, 'w') as f:
                    f.write(summary_report)
                
                print(f"✓ Summary report saved: {report_file}")
                
            else:
                print("⚠ Too few common columns to create global combined dataset")
                
        except Exception as e:
            print(f"❌ Error creating global combined dataset: {e}")
    
    # Final summary
    print(f"\n{'='*60}")
    print("CLEANING COMPLETE!")
    print(f"{'='*60}")
    print(f"Output directory: {output_dir}")
    print(f"- Individual city files: {output_dir / 'by_city'}")
    print(f"- Combined datasets: {output_dir / 'combined'}")
    print(f"- Reports: {output_dir / 'reports'}")
    print(f"Finished at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Execute the main function
if __name__ == "__main__":
    main()

=== TroTro Dataset Cleaning and Analysis ===
Starting analysis at: 2025-08-10 12:11:44
Skipping, found downloaded files in ".\trotro" (use force=True to force download)
✓ Dataset downloaded successfully
Using data directory: trotro/trotrolive-datasets/accra
Found 9 TXT files: ['agency.txt', 'calendar.txt', 'fare_attributes.txt', 'fare_rules.txt', 'routes.txt', 'shapes.txt', 'stops.txt', 'stop_times.txt', 'trips.txt']
Found 0 CSV files: []
Total data files to process: 9

Processing: agency.txt
Original TXT data loaded with tab separator: (91, 1)

Cleaning data for agency.txt...
  - Converted 'agency_id_agency_name_agency_url_agency_timezone' to datetime
  ✓ Cleaned data shape: (91, 1) → (91, 1)

--- Analysis for agency.txt ---
Shape: (91, 1)
Memory usage: 0.00 MB

Missing values:
agency_id_agency_name_agency_url_agency_timezone    91
dtype: int64

Data types:
datetime64[ns]    1
Name: count, dtype: int64

Sample data (first 5 rows):
  agency_id_agency_name_agency_url_agency_timezone
0  